In [7]:
!pip3 install torch
!pip3 install skorch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
  Using cached skorch-0.14.0-py3-none-any.whl (221 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [21]:

import os

import numpy as np
import pandas as pd
import utils
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer

from training_touchtales.estimator_helper_hc import EstimatorSelectionHelper

from training_touchtales.models import MODELS, PARAMS

from config_touchtale import TIME_INDEX, TIME_INTERVAL, WINDOW_SIZE, EXP_PARAMS, FS

from hiclass2 import metrics


INPUT_PICKLE_FILE = True
INPUT_DIR = 'COMBINED_DATA_TOUCHTALE'
INPUT_PICKLE_NAME = '_featurized_data.pk'
INPUT_LABEL_NAME = '_labels.pk'

SAVE_PICKLE_FILE = True
OUTPUT_DIR = 'RESULTS'
OUTPUT_PICKLE_NAME = 'results.pk'

SUBJECT_IDS = [x[:3] if 'p0' in x and 'cleaned' in x else '' for x in os.listdir('COMBINED_DATA_TOUCHTALE/')]
SUBJECT_IDS = list(filter(lambda a: a != '', SUBJECT_IDS))
print(SUBJECT_IDS)

LABEL_TYPES = ['pos', 'angle', 'acc', 'cw']

['p04', 'p03', 'p02', 'p00', 'p01']


In [34]:
def fit_helper(X, y, models=MODELS, params=PARAMS, n_jobs=-1, scoring={ "f1": make_scorer(metrics.f1), "prec" : make_scorer(metrics.precision), "recall": make_scorer(metrics.recall)}, cw_classes=None):
    helper = EstimatorSelectionHelper(models, params, cw_classes)
    helper.fit(X, y, scoring=scoring, n_jobs=n_jobs)
    try:
        scores = helper.score_summary(X, sort_by='max_f1')
        return helper, scores
    except ValueError:
        return -1, -1


def train(feature_dict, label_dict, label_types=LABEL_TYPES, pnum='p01'):

    # read the features back into a numpy array
    # bands = feature_dict[feature_types].to_numpy()
    # features = np.empty((bands.shape[0], bands.shape[1], 64, 64))
    # for band in range(len(feature_types)):
    #     band_arr = bands[:,band]
    #     for window in range(band_arr.shape[0]):
    #         band_im = np.array(band_arr[window])
    #         features[window, band, :, :] = band_im
    features = feature_dict.drop('timedelta', axis=1).to_numpy()
    X = features.astype(np.float32)

    res = {}
   
    res['pnum'] = pnum

    Y = label_dict.loc[label_dict['window_id'].isin(feature_dict['window_id'].unique())]

    # return

    LE = LabelEncoder() # transform string to class values
    LE.fit(Y['cw_mode'])

    y_cw_str = Y['cw_mode']
    y_cw = LE.transform(y_cw_str)

    for label_type in label_types: # train every label type
        utils.logger.info(f'Training label type {label_type}')

        y = np.array(list(zip(y_cw, Y[label_type])))
        y_str = np.array(list(zip(y_cw_str, Y[label_type] )))

        print('y\n', X.shape, y.shape)

        res['y_hc_' + label_type] = y_str

        helper, scores = fit_helper(X, y, cw_classes=len(LE.classes_))
        res['scores_hc_' + label_type] = scores
        del helper
        del scores

    return res

In [35]:
for i in range(0, 1):
        for window_size in EXP_PARAMS['WINDOW_SIZE']:
            if INPUT_PICKLE_FILE:
                participant_results = {}
                for subject_id in tqdm(SUBJECT_IDS):
                    # utils.logger.info(f'Training participant {subject_id}')

                    print(f'Training participant {subject_id}, {window_size}')

                    training_data_filename = subject_id + "_" + str(window_size) + 'ms' + INPUT_PICKLE_NAME
                    input_pickle_file_path = os.path.join(INPUT_DIR, training_data_filename)
                    input_label_file_path = os.path.join(INPUT_DIR, str(window_size) + 'ms' + INPUT_LABEL_NAME)

                    features = utils.load_pickle(pickled_file_path=input_pickle_file_path)
                    labels = utils.load_pickle(pickled_file_path=input_label_file_path)

                    utils.logger.info(f'Window size {window_size} - Iteration {i}')
                    training_results = train(features[subject_id], labels[subject_id], pnum=subject_id)

                    print(training_results)
                    # if SAVE_PICKLE_FILE:
                    #     iter_ = subject_id + "_" + str(i) + '_' + str(window_size) + 'ms_hc_cw_' + OUTPUT_PICKLE_NAME
                    #     os.makedirs(OUTPUT_DIR, exist_ok=True)
                    #     output_pickle_file_path = os.path.join(OUTPUT_DIR, iter_)
                    #     utils.pickle_data(data=training_results, file_path=output_pickle_file_path)

  0%|          | 0/5 [00:00<?, ?it/s]

Training participant p04, 500
y
 (5, 46) (5, 2)


/Users/poyuchen/Library/Python/3.9/lib/python/site-packages/skorch/utils.py:560: DeviceWarning: Requested to load data to CUDA but no CUDA devices are available. Loading on device "cpu" instead.
  warnings.warn(
/Users/poyuchen/Library/Python/3.9/lib/python/site-packages/skorch/utils.py:560: DeviceWarning: Requested to load data to CUDA but no CUDA devices are available. Loading on device "cpu" instead.
  warnings.warn(
/Users/poyuchen/Library/Python/3.9/lib/python/site-packages/skorch/utils.py:560: DeviceWarning: Requested to load data to CUDA but no CUDA devices are available. Loading on device "cpu" instead.
  warnings.warn(
/Users/poyuchen/Library/Python/3.9/lib/python/site-packages/skorch/utils.py:560: DeviceWarning: Requested to load data to CUDA but no CUDA devices are available. Loading on device "cpu" instead.
  warnings.warn(
/Users/poyuchen/Library/Python/3.9/lib/python/site-packages/skorch/net.py:2503: DeviceWarning: Setting self.device = cpu since the requested device (cud

ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/poyuchen/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/poyuchen/Desktop/UBC/Engineering-Physics/Fifth-Year/Summer/SPIN/emod-eegify.nosync/src/hiclass2/LocalClassifierPerParentNode.py", line 101, in fit
    super().fit(X, y, sample_weight=sample_weight)
  File "/Users/poyuchen/Desktop/UBC/Engineering-Physics/Fifth-Year/Summer/SPIN/emod-eegify.nosync/src/hiclass2/HierarchicalClassifier.py", line 124, in fit
    self._fit_digraph()
  File "/Users/poyuchen/Desktop/UBC/Engineering-Physics/Fifth-Yea